Nama: Feelouis Elfredo

Nim: 2502052726

Kelas: LA05

# ***BONUS POINT***

***The primary distinction between the BERT (Bidirectional Encoder Representations from Transformers) and GPT (Generative Pre-trained Transformer) models lies in their training approaches. BERT undergoes bidirectional training, processing input text in both directions simultaneously through masked language modeling. This methodology makes it well-suited for comprehending the context of a given sentence. Conversely, GPT is trained unidirectionally, exclusively processing from left to right, resulting in an autoregressive nature.***

# nomor 2C nyerah gabisa dirun 😢😢, yang 2D bisa dirun tapi harus nunggu 94x34min

# 2. [LO 1, LO 2, LO 3, LO4 35 points]
You are a data scientist tasked with developing a sentiment
analysis system for a hospital in Indonesia. This system aims to discern emotions from
questionnaire responses. You have access to various datasets containing information such as
Text and sentiment labels. These datasets are stored under the name “Emotion.csv”. The

a. [LO 1, LO 2, LO 3, LO 4, 5 points] Perform Preprocess and prepare Dataset step, describe
each step including some concepts in Natural Language Processing (NLP)

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
import nltk

#Splitting Data
from sklearn.model_selection import train_test_split

#Tokenizing Text Data
from transformers import AlbertTokenizer

#Model
from transformers import TFAlbertForSequenceClassification
from nltk.corpus import stopwords, wordnet
from string import punctuation

#Performance
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

#installation
!pip install transformer
!pip install sentencepiece

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


In [2]:
# Load the dataset
dt = pd.read_csv('Emotion.csv')
dt.head()

text    label
0                            i didnt feel humiliated  sadness
1  i can go from feeling so hopeless to so damned...  sadness
2   im grabbing a minute to post i feel greedy wrong    anger
3  i am ever feeling nostalgic about the fireplac...     love
4                               i am feeling grouchy    anger

In [3]:
shape = dt.shape

# printing shape
print("Shape = {}".format(shape))

Shape = (20000, 2)


In [4]:
# Check information about the dataset
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20000 non-null  object
 1   label   20000 non-null  object
dtypes: object(2)
memory usage: 312.6+ KB


In [5]:
# Check for missing values
dt.isnull().sum()

text     0
label    0
dtype: int64

In [6]:
# Check for duplicated rows
dt.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
19995    False
19996    False
19997    False
19998    False
19999    False
Length: 20000, dtype: bool

In [7]:
# Drop duplicates
dt = dt.drop_duplicates()

In [8]:
# Print column names
print(dt.columns)

Index(['text', 'label'], dtype='object')


b. [LO 1, LO 2, LO 3, LO 4, 5 points] Divide the dataset into training, validation, and testing
sets following these guidelines: allocate 70% of the data for training, 15% for validation,
and 15% for testing.

In [9]:
X = dt['text'].astype(str)
y = dt['label']

# Split the dataset into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp,
                                                test_size=0.5,
                                                random_state=42)


In [10]:
print(f"Training set size: {len(X_train)} samples")
print(f"Validation set size: {len(X_val)} samples")
print(f"Testing set size: {len(X_test)} samples")

Training set size: 13999 samples
Validation set size: 3000 samples
Testing set size: 3000 samples


c. [LO 1, LO 2, LO 3, LO 4 15 points] Develop the model based on Table 1. Train your model
based on the train dataset until achieving satisfactory accuracy.

In [11]:
dt = dt.copy()

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
def clean_text(text):
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    return text

dt['text'] = dt['text'].apply(clean_text)

In [13]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

train_encodings = tokenizer(list(X_train), truncation=True,
                            padding=True, max_length=128,
                            return_tensors='tf')

val_encodings = tokenizer(list(X_val), truncation=True,
                          padding=True, max_length=128,
                          return_tensors='tf')

test_encodings = tokenizer(list(X_test), truncation=True,
                           padding=True, max_length=128,
                           return_tensors='tf')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

In [14]:
albert_model = TFAlbertForSequenceClassification.from_pretrained('albert-base-v2')

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFAlbertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFAlbertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')


In [16]:
# model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)

In [17]:
# Compile the model (customize as needed)
albert_model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

In [18]:
# Display the model summary
albert_model.summary()

Model: "tf_albert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 albert (TFAlbertMainLayer)  multiple                  11683584  
                                                                 
 dropout_4 (Dropout)         multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 11685122 (44.58 MB)
Trainable params: 11685122 (44.58 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Training the model

In [19]:
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()
y_test = y_test.to_numpy()

In [20]:
history = albert_model.fit(X_train,
                            y_train,
                            validation_data = (X_val, y_val),
                            epochs = 6,
                            batch_size = 32)

Epoch 1/6


TypeError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_tf_utils.py", line 1640, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_file08gl_bep.py", line 37, in tf__run_call_with_unpacked_inputs
        retval_ = ag__.converted_call(ag__.ld(func), (ag__.ld(s

d. [LO 1, LO 2 & LO 3, LO 4 10 points] Performance analysis based on testing dataset (find
precision, recall, F1-Score and accuracy) of the model and analysis the result.

In [23]:
# Make predictions on the test set
test_predictions = albert_model.predict(
    x={'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']}
)

threshold = 0.5
binary_predictions = (test_predictions > threshold).astype(int)

# True labels (ground truth) for the test set
y_true = y_test


 1/94 [..............................] - ETA: 28:53

KeyboardInterrupt: 

In [ ]:
# Compute metrics
accuracy = accuracy_score(y_true, binary_predictions)
precision = precision_score(y_true, binary_predictions)
recall = recall_score(y_true, binary_predictions)
f1 = f1_score(y_true, binary_predictions)

In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(y_true, binary_predictions)

In [ ]:
# Print the results
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print("\nConfusion Matrix:")
print(conf_matrix)